In [2]:
!pip install --upgrade tensorflow
!pip install keras
!pip install pandas
!pip install numpy
# !pip install -q pyyaml h5py

import pandas as pd
import numpy as np
import re
import keras
import numpy as np
import tensorflow as tf
import math

from keras.layers import LSTM, GRU
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Dense, Bidirectional
from keras.layers import concatenate,Dropout

     |████████████████████████████████| 421.8MB 41kB/s 
     |████████████████████████████████| 3.9MB 51.4MB/s 
     |████████████████████████████████| 450kB 63.3MB/s 
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


Using TensorFlow backend.


In [1]:
from google.colab import drive
drive.mount("/content/drive")


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
class MyModel2(tf.keras.Model):
    def __init__(self, vocab_size):
        super(MyModel2, self).__init__()
        self.embedding_layer = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=(512),embeddings_initializer="glorot_normal")
        self.bidirectional_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, return_sequences=True))
        self.dense_layer = tf.keras.layers.Dense(1, activation="sigmoid")

    def __call__(self, q, p, o):

        ## embedding layer
        queryEmbedding = self.embedding_layer(q)
        predEmbedding = self.embedding_layer(p)
        objEmbedding = self.embedding_layer(o)        

        ## reshaping before feeding to Bi-LSTM
        queryEmbedding = tf.reshape(queryEmbedding, (1, queryEmbedding.shape[0], queryEmbedding.shape[1]))
        predEmbedding = tf.reshape(predEmbedding, (1, predEmbedding.shape[0], predEmbedding.shape[1]))
        objEmbedding = tf.reshape(objEmbedding, (1, objEmbedding.shape[0], objEmbedding.shape[1]))
        
        ## Bi-LSTM
        queryEmbedding = self.bidirectional_lstm(queryEmbedding)
        predEmbedding = self.bidirectional_lstm(predEmbedding)
        objEmbedding = self.bidirectional_lstm(objEmbedding)
        
        ## taking the first and last outputs of lstm layer 
        ## first -> right context
        ## last -> left context
        queryConcat = tf.concat([queryEmbedding[:,0,:512],queryEmbedding[:,49,512:]],axis=1)
        predConcat = tf.concat([predEmbedding[:,0,:512],predEmbedding[:,49,512:]],axis=1)
        objConcat = tf.concat([objEmbedding[:,0,:512],objEmbedding[:,49,512:]],axis=1)
        

        ## (q + p).oT = target
        customTarget = queryConcat+predConcat           
        customTarget=tf.matmul(customTarget,tf.transpose(objConcat))
        
        return customTarget



In [0]:
## homogeneous encoding for all the queries,predicates, objects
maxLengthPadding = 50


## split the words in predicates and objects
def splitCamelCasing(camelCasedWord):
    camelCaseSplit = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', camelCasedWord)).split()
    return camelCaseSplit
facts = pd.read_excel('/content/drive/My Drive/Independent Study/output.xlsx')
facts = facts.drop(["id", "qid", "imp", "rel", "en_id"], axis=1)



In [0]:
## To give a higher importance to a higher utility value
def invertRanks(rankArray):
  maximum = 5 
  for i in range(len(rankArray)):
    if(rankArray[i]==0):
      rankArray[i]=1/maximum
    else:
      rankArray[i]=rankArray[i]/maximum
  return rankArray


ENCODING_DIM = 50
maxRank = 0
y = invertRanks(list(facts['utility'].values))


In [0]:
embeddings_dictionary = {}

## Creates an encoding for the entire wordList
def encode_sequence(wordList):
    encodedValue = []  ## dimension of embedding
    for word in wordList:
        encodedValue += [embeddings_dictionary[word]]
    ## padding the remaining values to create a homogeneous encoding
    encodedValue += [0] * (maxLengthPadding - len(encodedValue))
    return encodedValue


################################################# Preparing the Embedding Space / Dictionary of Words ############################################


############# 1. The query encoding ###############

x = []
queryWords = []
adjustedPredicates = []
objWords = []

for i in range(len(facts["pred"])):
    ## splitting the query
    tempName = re.sub('[^a-zA-Z0-9 \n\.]', '', facts["query"].values[i].strip())
    queryWords += tempName.lower().strip().split(" ")

    ## splitting the object
    oW = facts["obj"].values[i].lower().strip()
    if ('<' in oW and '>' in oW and 'dbp' in oW):
        oW = oW.split(":")
        oW = oW[1].split(">")
        oW = oW[0].split('_')
    elif ('www' in oW):
        oW = oW.split(".")
    else:
        oW = oW.split(" ")

    ## for removing special characters
    oW = [re.sub('[^a-zA-Z0-9 \n\.]', '', x) for x in oW]
    objWords += oW
    
    #splitting the predicates
    pred = facts["pred"].values[i].lower().split(":")
    pred = pred[1].split(">")
    pred = splitCamelCasing(pred[0])
    pred = [re.sub('[^a-zA-Z0-9 \n\.]', '', x) for x in pred]
    adjustedPredicates += pred

totalWords = queryWords + adjustedPredicates + objWords
totalWords = set(totalWords)
vocab_size = len(totalWords)

## creating the dictionary
for i, x in enumerate(totalWords):
    embeddings_dictionary[x] = i



In [0]:
queryWordEncodings = []
predWordEncodings = []
objWordEncodings = []


for i in range(len(facts["pred"].values)):

    name = re.sub('[^a-zA-Z0-9 \n\.]', '', facts["query"].values[i])
    queryWords = name.lower().strip().split(" ")
    queryWordEncodings += [encode_sequence(queryWords)]
    
    ## similar to creating the dictionary, but this time it encodes the words using the values from dictionary
    oW = facts["obj"].values[i].lower().strip()
    if ('<' in oW and '>' in oW and 'dbp' in oW):
        oW = oW.split(":")
        oW = oW[1].split(">")
        oW = oW[0].split('_')
    elif ('www' in oW):
        oW = oW.split(".")
    else:
        oW = oW.split(" ")

    ## for removing special characters
    oW = [re.sub('[^a-zA-Z0-9 \n\.]', '', x) for x in oW]
    oW = encode_sequence(oW)
    objWordEncodings += [oW]

    ## encoding the words of predicates
    pred = facts["pred"].values[i].lower().split(":")
    pred = pred[1].split(">")
    pred = splitCamelCasing(pred[0])
    pred = [re.sub('[^a-zA-Z0-9 \n\.]', '', x) for x in pred]
    predWordEncodings += [encode_sequence(pred)]

# print(len(queryWordEncodings[0]),len(objWordEncodings[0]),len(predWordEncodings[0]))
    


In [0]:
## defining custom model parameters as per tensorflow2 guidelines

model_2 = MyModel2(vocab_size)
loss_obj = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.MeanSquaredError(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.MeanSquaredError(name='test_accuracy')



In [0]:
## x_tr : training x -> contains the query data, predicate data, object data
## y_tr : training y -> contains the normalized targets
def train_step(x_tr,y_tr):
    qE = x_tr[0]
    pE = x_tr[1]
    oE = x_tr[2]
    batch_loss = 0
    with tf.GradientTape() as tape:
        for i in range(len(qE)):
            q= tf.convert_to_tensor(qE[i], dtype=tf.int32)
            p = tf.convert_to_tensor(pE[i], dtype=tf.int32)
            o = tf.convert_to_tensor(oE[i], dtype=tf.int32)
            predictions = model_2(q, p, o)
            loss = loss_obj(y_tr[i],predictions)
            batch_loss+=loss
        gradients = tape.gradient(batch_loss/len(qE), model_2.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model_2.trainable_variables))
    return batch_loss
  
  
## test data set
def test_step(x_ts,y_ts):
    queryWordEncodings = x_ts[0]
    predWordEncodings = x_ts[1]
    objWordEncodings = x_ts[2]
    loss_mean_test = 0
    # print("--------------------------------------------------------x--------------------------------------------------------x-----------------------------------------------")  
    for i in range(len(queryWordEncodings)):
        q= tf.convert_to_tensor(queryWordEncodings[i], dtype=tf.int32)
        p = tf.convert_to_tensor(predWordEncodings[i], dtype=tf.int32)
        o = tf.convert_to_tensor(objWordEncodings[i], dtype=tf.int32)
        predictions = model_2(q,p,o)
        t_loss = loss_obj(y_ts[i], predictions)
        loss_mean_test+=t_loss
        test_loss(t_loss)
        test_accuracy(y_test[i], predictions)
        # print(i)
    return loss_mean_test/len(queryWordEncodings)


In [22]:
from sklearn.utils import shuffle

EPOCHS = 10

data_len = len(queryWordEncodings)

queryWordEncodingsTrain,predWordEncodingsTrain,objWordEncodingsTrain,y_train = shuffle(queryWordEncodings[:data_len],objWordEncodings[:data_len],predWordEncodings[:data_len],y[:data_len])
queryWordEncodingsTest,predWordEncodingsTest,objWordEncodingsTest,y_test = shuffle(queryWordEncodings[data_len:],objWordEncodings[data_len:],predWordEncodings[data_len:],y[data_len:])



batch_size = 16

num_batches_train = math.floor(data_len/batch_size)

test_size = len(queryWordEncodings)
num_batches_test = math.floor(test_size/batch_size)


print("Num Batches Train:",num_batches_train)
print(data_len/batch_size)
print(len(y_train))

Num Batches Train: 81
81.8125
1309


In [23]:

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  loss_per_epoch = 0

  for i in range(num_batches_train):
    x_train = [queryWordEncodingsTrain[i*batch_size:(i+1)*batch_size],predWordEncodingsTrain[i*batch_size:(i+1)*batch_size],objWordEncodingsTrain[i*batch_size:(i+1)*batch_size]]
    loss1 = train_step(x_train,y_train[i*batch_size:(i+1)*batch_size])
    loss_per_epoch += loss1
  x_train = [queryWordEncodingsTrain[num_batches_train*batch_size:],predWordEncodingsTrain[num_batches_train*batch_size:],objWordEncodingsTrain[num_batches_train*batch_size:]]
  loss1 = train_step(x_train,y_train[num_batches_train*batch_size:])
  loss_per_epoch += loss1
  print("Epoch Loss for epoch "+str(epoch)+": ",loss_per_epoch/(num_batches_train+1) )

  # for i in range(num_batches_test):
  #   x_test = [queryWordEncodingsTest[i*batch_size:(i+1)*batch_size],predWordEncodingsTest[i*batch_size:(i+1)*batch_size],objWordEncodingsTest[i*batch_size:(i+1)*batch_size]]
  #   loss2 = test_step(x_test,y_test[i*batch_size:(i+1)*batch_size])
  #   print(loss2)  

  # x_test = [queryWordEncodingsTest[num_batches_test*batch_size:],predWordEncodingsTest[num_batches_test*batch_size:],objWordEncodingsTest[num_batches_test*batch_size:]]
  # loss2 = test_step(x_test,y_test[num_batches_test*batch_size:])
  # print(loss2)



Epoch Loss for epoch 0:  tf.Tensor(0.4818991, shape=(), dtype=float32)
Epoch Loss for epoch 1:  tf.Tensor(0.23604867, shape=(), dtype=float32)
Epoch Loss for epoch 2:  tf.Tensor(0.13714768, shape=(), dtype=float32)
Epoch Loss for epoch 3:  tf.Tensor(0.11405318, shape=(), dtype=float32)
Epoch Loss for epoch 4:  tf.Tensor(0.18292435, shape=(), dtype=float32)
Epoch Loss for epoch 5:  tf.Tensor(0.19767022, shape=(), dtype=float32)
Epoch Loss for epoch 6:  tf.Tensor(0.14313562, shape=(), dtype=float32)
Epoch Loss for epoch 7:  tf.Tensor(0.07201068, shape=(), dtype=float32)
Epoch Loss for epoch 8:  tf.Tensor(0.0597663, shape=(), dtype=float32)
Epoch Loss for epoch 9:  tf.Tensor(0.05771786, shape=(), dtype=float32)


In [24]:
model_2.save_weights('/content/drive/My Drive/Independent Study/MiniBatch_URI_Data')

model_load = MyModel2(vocab_size)
model_load.load_weights('/content/drive/My Drive/Independent Study/MiniBatch_URI_Data')

In [0]:
## grouping the dataset by query to get the ndcg scores

def modifiedInvertRanks(rankArray):
  for rank in rankArray:
    maximum = 5 
    for j in range(len(rank)):
      if(rank[j]==0):
        rank[j]=1/maximum
      else:
        rank[j]=rank[j]/maximum
    
  return rankArray

modified_y = []
groupedResults = facts.groupby('query')

for name, group in groupedResults:
    g = list(group['utility'].values)
    # print(len(g))
    modified_y.append(g)
modified_y = modifiedInvertRanks(modified_y)


queryWordEncodings = []
predWordEncodings = []
objWordEncodings = []

predictedQueryRanks = []
groundTruthRanks = []

count = 0

for query, group in groupedResults:

    ## name is the query used for grouping
    query = re.sub('[^a-zA-Z0-9 \n\.]', '', query)
    queryWords = query.lower().strip().split(" ")
    q = encode_sequence(queryWords)
    predictedRanksPerQuery = []
    groundTruthRanksPerQuery = []

    ## encoding predicate-objects
    
    for i in range(len(group["obj"].values)):
        oW = group["obj"].values[i].lower().strip()
        if ('<' in oW and '>' in oW and 'dbp' in oW):
            oW = oW.split(":")
            oW = oW[1].split(">")
            oW = oW[0].split('_')
        elif ('www' in oW):
            oW = oW.split(".")
        else:
            oW = oW.split(" ")

        ## for removing special characters
        oW = [re.sub('[^a-zA-Z0-9 \n\.]', '', x) for x in oW]
        o = encode_sequence(oW)
        
        pred = group["pred"].values[i].lower().split(":")
        pred = pred[1].split(">")
        pred = splitCamelCasing(pred[0])
        pred = [re.sub('[^a-zA-Z0-9 \n\.]', '', x) for x in pred]
        p = encode_sequence(pred)
        
        q= tf.convert_to_tensor(q, dtype=tf.int32)
        p = tf.convert_to_tensor(p, dtype=tf.int32)
        o = tf.convert_to_tensor(o, dtype=tf.int32)
        
        predictions = model_load(q, p, o)
        predictedRanksPerQuery.append(tf.math.round(predictions*5).numpy().tolist()[0][0])

    g_t = [x*5 for x in modified_y[count]]
    groundTruthRanks.append(g_t)
    predictedQueryRanks.append(predictedRanksPerQuery)
    count+=1


    

In [26]:
def dcg_score(y_true, y_score, k=5):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    gain = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gain / discounts)


def ndcg_score(ground_truth, predictions, k=5):
    scores = []
    for y_true, y_score in zip(ground_truth, predictions):
        actual = dcg_score(y_true, y_score, k)
        best = dcg_score(y_true, y_true, k)
        score = float(actual) / float(best)
        scores.append(score)


    return np.mean(scores)

print("NDCG@5:",ndcg_score(groundTruthRanks,predictedQueryRanks,k=5))
print("NDCG@10:",ndcg_score(groundTruthRanks,predictedQueryRanks,k=10))



NDCG@5: 0.9544463768893355
NDCG@10: 0.9681096628514892


In [27]:
## to check the embedding space

for x in embeddings_dictionary.keys():
  print(x)


tacky
repteam
ostermundigen
great
operant
give
cornwall
mediagroup
districts
hubairport
elia
klerk
firstholder
a.
dont
claudio
elijah
americans
mnage
greg
firstminister
kentucky
spouse
hooligans
rugby
bulgaria
football
nationalanthem
oklahoma
methodism
anchorage
iso
donlan
computer
presbyterian
outer
3
ohio
manufacturing
v
album
thomas
nudisco
youth
governingbody
libre
philippines
world
metropolitan
geotld
commissioner
center
illinois
bremen
ammonium
salsa
affairs
occupation
canadian
tokyo
mandela
governor
kalmar
stands
loeb
precedence
mp3
krone
durango
inuvialuktun
deathplace
germany
german
jimmy.
minnesota
climber
legaljuris
sri
places
berlin
nicks
nature
baby
pop
eloan
western
anthem
eastern
spanish
andress
pacific
cecil
european
hubs
sports
palmetto
brimson
yupik
heart
journalist
guarantee
travel
debutleague
greater
gyude
or
ashley
politician
blankinfo
environment
accreditation
carl
house
ringtone
mills
teentoday.co.uk
u.s.
espresso
influenced
start
gwichin
vietnam
milton
romanova